Download the test dataset from Kaggle and merge it with your current data set.

Split your dataset into a training dataset from 01.07.2013 to 31.07.2017, a validation dataset from 01.08.2017 to 31.07.2018, and the test set from 01.08.2018 to 31.07.2019.